In [44]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [80]:
def build_prompt(query, search_results):

    context_template = """
Q: {question}
A: {text}
""".strip()
    
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""

    for doc in search_results:
        context = context_template + "\n\n" 
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt
        

In [46]:
from groq import Groq
client = Groq()

In [47]:
def llm(prompt):
    response = client.chat.completions.create(
    model="compound-beta",
    messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [48]:
from elasticsearch import Elasticsearch

In [49]:
ec_client = Elasticsearch("http://localhost:9200")

In [50]:
ec_client.info()

ObjectApiResponse({'name': '089eef0efac9', 'cluster_name': 'docker-cluster', 'cluster_uuid': '2qzVOdt9TSuVMYeHplDoWA', 'version': {'number': '9.0.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '0a58bc1dc7a4ae5412db66624aab968370bd44ce', 'build_date': '2025-05-28T10:06:37.834829258Z', 'build_snapshot': False, 'lucene_version': '10.1.0', 'minimum_wire_compatibility_version': '8.18.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'})

In [51]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions-homework"

ec_client.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions-homework/nqBAt4HoR_GAp1v4D_9ZcQ] already exists')

In [52]:
from tqdm.auto import tqdm

In [53]:
for doc in tqdm(documents):
    ec_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [67]:
query1 = "How do execute a command on a Kubernetes pod?"
query2 = "How do copy a file to a Docker container?"

In [77]:
def elastic_search(query):
    
    search_query = {
    "size": 3, #number of results 
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],  # ^ means boost
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}
    response = ec_client.search(index=index_name, body=search_query)
    print(response)
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

In [78]:
elastic_search(query2)

{'took': 15, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1360, 'relation': 'eq'}, 'max_score': 73.563896, 'hits': [{'_index': 'course-questions-homework', '_id': 'iANad5cBQI3gdSavXxyj', '_score': 73.563896, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions-homework', '_id': '1ANXd5cBQI3gdSavtBjl', '_score': 73.563896, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Launch the container im

In [59]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [82]:
rag_response = rag(query2)

{'took': 11, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1360, 'relation': 'eq'}, 'max_score': 73.563896, 'hits': [{'_index': 'course-questions-homework', '_id': 'iANad5cBQI3gdSavXxyj', '_score': 73.563896, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions-homework', '_id': '1ANXd5cBQI3gdSavtBjl', '_score': 73.563896, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker

In [83]:
len(rag_response)

845

In [86]:
pip install tiktoken



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [87]:
encoding = tiktoken.encoding_for_model("compound-beta")

NameError: name 'tiktoken' is not defined